In [ ]:
%pip install -qqqU PyPDF2 langchain langchain-community matplotlib pypdf langchain-qdrant pandas tqdm

In [ ]:
from src.utils import load_docs_from_json

documents = load_docs_from_json('regulamento-semantic.json')

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = """
A partir  o artigo de regulamento abaixo, gere 5 perguntas e respostas referentes a ela.

{context}


Exemplo de perguntas que alunos fariam:

Qual o procedimento pra pedir trancamento de curso?
O que preciso fazer para pedir pra ver o registro de frequencia?
Até quando posso realizar matricula em TCC?

Escreva as perguntas nesse formato: 

Q: Gostaria de saber como devo realizar o requerimento do aproveitamento de disciplinas
A: A solicitação do aproveitamento de estudos deve ser solicitada no SIGAA pelo interessado, e pode procurar sua coordenação em caso de dúvidas. Caso sejam apenas disciplinas ELETIVAS cursadas na UFRN ou REEXAME de um processo físico arquivado no Segia, o interessado deve realizar a solicitação na Coordenadoria de Atendimento da Prograd.
Q: Perdi o prazo final para o cadastramento na UFRN, ainda posso realizar o cadastramento?
A: O cadastramento para candidatos convocados pela UFRN somente ocorre caso toda a Documentação seja entregue no período estipulado em Edital. Não é possível fazê-lo posteriormente ao prazo estipulado.
Q: Com qual nota sou considerado aprovado na UFRN?
A: Para ser considerado aprovado nos cursos de graduação da UFRN, o aluno deve ter média parcial igual ou superior a 6,0, com rendimento igual ou superior a 4,0 em todas as unidades
"""

prompt_template = PromptTemplate.from_template(prompt)

In [ ]:
from langchain_aws import BedrockLLM

model_id = "meta.llama3-70b-instruct-v1:0"

custom_llm = BedrockLLM(
    model_id=model_id,  # ARN like 'arn:aws:bedrock:...' obtained via provisioning the custom model
    model_kwargs={"temperature": 0.8},
    streaming=True,
)

In [ ]:
artigo = documents[130].page_content
prompt = prompt_template.format(context=artigo)
response = custom_llm.invoke(prompt)

print(artigo, '\n\n', response)

In [ ]:
import re
import pandas as pd
from tqdm import tqdm
from time import sleep

# Regex para capturar perguntas e respostas
regex = r"Q:\s(.*?)\nA:\s(.*?)\n(?=Q:|$)"

dados = {
    "NumeroArtigo": [],
    "TextoArtigo": [],
    "Pergunta": [],
    "Resposta": [],
}

for doc in tqdm(documents[50:]):
    try:
        prompt = prompt_template.format(context=doc.page_content)
        response = custom_llm.invoke(prompt)

        # Usando findall para capturar as perguntas e respostas
        matches = re.findall(regex, response, re.DOTALL)

        # Criando o dicionário
        dados["NumeroArtigo"] += [doc.metadata["Artigo"] for _ in matches]
        dados["TextoArtigo"] += [doc.page_content for _ in matches]
        dados["Pergunta"] += [match[0].strip() for match in matches]
        dados["Resposta"] += [match[1].strip() for match in matches]

        sleep(5)
    except:
        sleep(60)
        

In [ ]:
# Convertendo para um DataFrame
df = pd.DataFrame(dados)

df.to_csv("potiguana_qa_p2.csv")

# Exibindo o DataFrame
df.head()